In [5]:
import os
import time
import math
import random
import pickle
import numpy as np
import tensorflow as tf
import multiprocess as mp
from libs.TicTacToe import TicTacToe
from libs.model import create_az_model
from libs.MCTS import MCTS
from libs.Trainer import Trainer

In [6]:
n_cores = mp.cpu_count()
print(f"[Number of cores]: {n_cores}")

[Number of cores]: 16


## Tic-Tac-Toe engine

In [7]:
state = np.array([0., 0., -1., -1., 1., -1., 1., 1., 0.])
TicTacToe().get_available_actions(state)

array([1., 1., 0., 0., 0., 0., 0., 0., 1.])

In [8]:
ttt = TicTacToe()

while ttt.status=="Ongoing":

    valid_moves = ttt.next_moves()
    new_board = random.choice(valid_moves)
    ttt.play(new_board)
    print(np.reshape(ttt.board, (3,3)),"\n")
    if len(valid_moves)==0:
        break

print("[Winner]:",ttt.winner)

[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]] 

[[ 0.  0.  0.]
 [ 1.  0. -1.]
 [ 0.  0.  0.]] 

[[ 0.  0.  0.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]] 

[[ 0.  0. -1.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]] 

[[ 0.  1. -1.]
 [ 1.  0. -1.]
 [ 0.  0.  1.]] 

[[ 0.  1. -1.]
 [ 1.  0. -1.]
 [-1.  0.  1.]] 

[[ 0.  1. -1.]
 [ 1.  1. -1.]
 [-1.  0.  1.]] 

[[-1.  1. -1.]
 [ 1.  1. -1.]
 [-1.  0.  1.]] 

[[-1.  1. -1.]
 [ 1.  1. -1.]
 [-1.  1.  1.]] 

[Winner]: 1


## Create policy and probability model

In [9]:
model = create_az_model()

In [10]:
model.summary()

Model: "tictactoe_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 9)]          0           []                               
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 9)           0           ['input_1[0][0]']                
 da)                                                                                              
                                                                                                  
 tf.cast (TFOpLambda)           (None, 9)            0           ['tf.__operators__.add[0][0]']   
                                                                                                  
 tf.one_hot (TFOpLambda)        (None, 9, 3)         0           ['tf.cast[0][0]']  

In [11]:
tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Monte Carlo Tree Search

In [12]:
mcts = MCTS(game=TicTacToe(), n_simulations=100)

In [13]:
state = np.array([0., 0., -1., -1., 1., -1., 1., 1., 0.])

In [14]:
root = mcts.run(model=model, state=state, player=1)

1/1 [==============================] - 0s 17ms/step


In [15]:
root

State:
[ 0.  0. -1. -1.  1. -1.  1.  1.  0.]
Player:1
Value:0.96
Leaf:False
Visits:100
Children:
 -0: prior=0.3333333333333333
 -1: prior=0.3333333333333333
 -8: prior=0.3333333333333333

# Generating the first dataset

In [16]:
# speed test on 100 games
start_time = time.time()
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=None)
dataset = trainer.create_dataset(number_of_games=100, temperature=1)
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 23.375245571136475 seconds


In [17]:
def parallel_fn(n):
    from libs.Trainer import Trainer
    from libs.TicTacToe import TicTacToe
    from libs.MCTS import MCTS
    N_GAMES_PER_JOB = 250
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=None)
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=1)
    return game_batch

In [18]:
start_time = time.time()
ctx = mp.get_context("spawn")
pool = ctx.Pool(n_cores)
dataset = pool.map(parallel_fn, range(n_cores))
print("Running time: %s seconds" % (time.time() - start_time))

Running time: 120.05786895751953 seconds


In [20]:
N_GAMES_PER_JOB = 250
filename = str(N_GAMES_PER_JOB*n_cores)+"_tictactoe_temperature_1.pickle"

flatten_dataset = []
for batch in dataset:
    flatten_dataset += batch
    
with open(filename, "wb") as fp:
    pickle.dump(flatten_dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)

# 1st training

In [21]:
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=create_az_model())

In [22]:
with open(filename, 'rb') as handle:
    dataset = pickle.load(handle)

In [23]:
fit_history = trainer.train(dataset, n_epochs=1000, batch_size=30, learning_rate=1e-2)

Epoch 1/1000
870/870 [==============================] - 5s 3ms/step - loss: 2.2635 - policy_loss: 1.5550 - value_loss: 0.6090 - val_loss: 2.0708 - val_policy_loss: 1.4680 - val_value_loss: 0.4938 - lr: 0.0100
Epoch 2/1000
870/870 [==============================] - 3s 3ms/step - loss: 2.0459 - policy_loss: 1.4318 - value_loss: 0.4925 - val_loss: 2.0321 - val_policy_loss: 1.4105 - val_value_loss: 0.4880 - lr: 0.0100
Epoch 3/1000
870/870 [==============================] - 3s 3ms/step - loss: 2.0345 - policy_loss: 1.4168 - value_loss: 0.4792 - val_loss: 1.9926 - val_policy_loss: 1.3946 - val_value_loss: 0.4540 - lr: 0.0100
Epoch 4/1000
870/870 [==============================] - 3s 3ms/step - loss: 2.0213 - policy_loss: 1.4075 - value_loss: 0.4689 - val_loss: 1.9781 - val_policy_loss: 1.3962 - val_value_loss: 0.4359 - lr: 0.0100
Epoch 5/1000
870/870 [==============================] - 3s 3ms/step - loss: 2.0012 - policy_loss: 1.4007 - value_loss: 0.4578 - val_loss: 1.9144 - val_policy_loss: 

Epoch 40/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7223 - policy_loss: 1.3156 - value_loss: 0.3686 - val_loss: 1.6893 - val_policy_loss: 1.2958 - val_value_loss: 0.3555 - lr: 1.2907e-04
Epoch 41/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7214 - policy_loss: 1.3161 - value_loss: 0.3673 - val_loss: 1.6888 - val_policy_loss: 1.2955 - val_value_loss: 0.3554 - lr: 1.0000e-04
Epoch 42/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7209 - policy_loss: 1.3149 - value_loss: 0.3682 - val_loss: 1.6885 - val_policy_loss: 1.2956 - val_value_loss: 0.3553 - lr: 1.0000e-04
Epoch 43/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7193 - policy_loss: 1.3155 - value_loss: 0.3662 - val_loss: 1.6886 - val_policy_loss: 1.2956 - val_value_loss: 0.3555 - lr: 1.0000e-04
Epoch 44/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7205 - policy_loss: 1.3151 - value_loss: 0.3679 - val_loss: 1.687

870/870 [==============================] - 3s 3ms/step - loss: 1.7117 - policy_loss: 1.3126 - value_loss: 0.3662 - val_loss: 1.6819 - val_policy_loss: 1.2943 - val_value_loss: 0.3547 - lr: 1.0000e-04
Epoch 79/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7126 - policy_loss: 1.3131 - value_loss: 0.3668 - val_loss: 1.6817 - val_policy_loss: 1.2941 - val_value_loss: 0.3548 - lr: 1.0000e-04
Epoch 80/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7115 - policy_loss: 1.3129 - value_loss: 0.3659 - val_loss: 1.6817 - val_policy_loss: 1.2938 - val_value_loss: 0.3552 - lr: 1.0000e-04
Epoch 81/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7107 - policy_loss: 1.3125 - value_loss: 0.3657 - val_loss: 1.6815 - val_policy_loss: 1.2942 - val_value_loss: 0.3548 - lr: 1.0000e-04
Epoch 82/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7092 - policy_loss: 1.3124 - value_loss: 0.3643 - val_loss: 1.6815 - val_policy

870/870 [==============================] - 3s 3ms/step - loss: 1.7041 - policy_loss: 1.3104 - value_loss: 0.3646 - val_loss: 1.6784 - val_policy_loss: 1.2935 - val_value_loss: 0.3558 - lr: 1.0000e-04
Epoch 117/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7065 - policy_loss: 1.3110 - value_loss: 0.3665 - val_loss: 1.6783 - val_policy_loss: 1.2932 - val_value_loss: 0.3562 - lr: 1.0000e-04
Epoch 118/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7044 - policy_loss: 1.3109 - value_loss: 0.3646 - val_loss: 1.6776 - val_policy_loss: 1.2932 - val_value_loss: 0.3556 - lr: 1.0000e-04
Epoch 119/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7046 - policy_loss: 1.3105 - value_loss: 0.3653 - val_loss: 1.6780 - val_policy_loss: 1.2935 - val_value_loss: 0.3558 - lr: 1.0000e-04
Epoch 120/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7038 - policy_loss: 1.3107 - value_loss: 0.3644 - val_loss: 1.6780 - val_po

870/870 [==============================] - 3s 3ms/step - loss: 1.6986 - policy_loss: 1.3098 - value_loss: 0.3628 - val_loss: 1.6757 - val_policy_loss: 1.2930 - val_value_loss: 0.3567 - lr: 1.0000e-04
Epoch 155/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7003 - policy_loss: 1.3099 - value_loss: 0.3644 - val_loss: 1.6759 - val_policy_loss: 1.2933 - val_value_loss: 0.3566 - lr: 1.0000e-04
Epoch 156/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7010 - policy_loss: 1.3098 - value_loss: 0.3652 - val_loss: 1.6759 - val_policy_loss: 1.2929 - val_value_loss: 0.3571 - lr: 1.0000e-04
Epoch 157/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.7001 - policy_loss: 1.3105 - value_loss: 0.3637 - val_loss: 1.6752 - val_policy_loss: 1.2928 - val_value_loss: 0.3565 - lr: 1.0000e-04
Epoch 158/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.6988 - policy_loss: 1.3105 - value_loss: 0.3625 - val_loss: 1.6754 - val_po

870/870 [==============================] - 3s 3ms/step - loss: 1.6938 - policy_loss: 1.3083 - value_loss: 0.3619 - val_loss: 1.6721 - val_policy_loss: 1.2926 - val_value_loss: 0.3559 - lr: 1.0000e-04
Epoch 193/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.6949 - policy_loss: 1.3079 - value_loss: 0.3634 - val_loss: 1.6729 - val_policy_loss: 1.2930 - val_value_loss: 0.3564 - lr: 1.0000e-04
Epoch 194/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.6939 - policy_loss: 1.3091 - value_loss: 0.3613 - val_loss: 1.6727 - val_policy_loss: 1.2926 - val_value_loss: 0.3566 - lr: 1.0000e-04
Epoch 195/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.6960 - policy_loss: 1.3096 - value_loss: 0.3629 - val_loss: 1.6723 - val_policy_loss: 1.2928 - val_value_loss: 0.3560 - lr: 1.0000e-04
Epoch 196/1000
870/870 [==============================] - 3s 3ms/step - loss: 1.6959 - policy_loss: 1.3089 - value_loss: 0.3636 - val_loss: 1.6726 - val_po

In [24]:
trainer.model.predict(np.array([[0., 0., -1., -1., 1., -1., 1., 1., 0.]]))

1/1 [==============================] - 0s 171ms/step


(array([[7.3029539e-03, 9.9688314e-02, 4.8711382e-09, 3.8707190e-06,
         1.1328524e-06, 6.5809280e-07, 7.0552865e-08, 1.1762501e-06,
         8.9300179e-01]], dtype=float32),
 array([[0.99997973]], dtype=float32))

In [25]:
trainer.model.predict(np.array([[0., 0., 0., 0., -1., 1., -1., 0., 0.]]))

1/1 [==============================] - 0s 28ms/step


(array([[7.0015445e-02, 8.6133733e-02, 5.7866484e-01, 9.6366681e-02,
         1.1903725e-04, 3.7989702e-07, 3.1434181e-06, 7.8452431e-02,
         9.0244174e-02]], dtype=float32),
 array([[-0.910473]], dtype=float32))

In [26]:
trainer.save_model('saved_model/alpha_zero_model_first_training_validation')

INFO:tensorflow:Assets written to: saved_model/alpha_zero_model_first_training_validation\assets


INFO:tensorflow:Assets written to: saved_model/alpha_zero_model_first_training_validation\assets


# Generating the self-playing dataset (2nd)

In [27]:
loaded_model = tf.keras.models.load_model("saved_model/alpha_zero_model_first_training_validation/")

In [28]:
# speed test on 10 games
start_time = time.time()
trainer = Trainer(game = TicTacToe, mcts = MCTS, model=loaded_model)
dataset = trainer.create_dataset(number_of_games=10, temperature=1)
print("Running time: %s seconds" % (time.time() - start_time))

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 11ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


...

1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 30ms/step


KeyboardInterrupt: 

In [ ]:
# 1175s -> ~30m for 100
# 25*0.5 ~ 12hs

In [29]:
N_ROUNDS = 20
N_GAMES_PER_JOB = 25

In [30]:
def parallel_fn(job_n, n):
    
    from libs.Trainer import Trainer
    from libs.TicTacToe import TicTacToe
    from libs.MCTS import MCTS
    import tensorflow as tf
    N_GAMES_PER_JOB = 25
    
    model = tf.keras.models.load_model("saved_model/tmp_model")
    
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=model)
    
    game_batch = trainer.create_dataset(number_of_games=N_GAMES_PER_JOB, temperature=1.0/n)
    
    return game_batch

In [31]:
#stored previous self-plays
filename_data = "4000_tictactoe_temperature_1_2nd_round.pickle"
if os.path.isfile(filename_data):
    with open(filename_data, 'rb') as handle:
        self_play_dataset = pickle.load(handle)
    print(f"Reusing {filename_data}")
else:
    self_play_dataset = []
    print(f"Starting from empty self-play data")

model_tmp_filename = "saved_model/tmp_model"
if os.path.isdir(model_tmp_filename):
    model = tf.keras.models.load_model(model_tmp_filename)
    print(f"Starting with {model_tmp_filename}")
else:
    model = tf.keras.models.load_model("saved_model/alpha_zero_model_first_training_validation/")
    print(f"Starting with saved_model/alpha_zero_model_first_training_validation/")

model.save(model_tmp_filename)

Starting from empty self-play data
Starting with saved_model/alpha_zero_model_first_training_validation/


INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


INFO:tensorflow:Assets written to: saved_model/tmp_model\assets


In [ ]:

for n in range(N_ROUNDS):
    
    start_time = time.time()
    
    ctx = mp.get_context("spawn")
    
    pool = ctx.Pool(n_cores)
    
    args = list(zip(range(n_cores),[n+1]*n_cores))
    
    round_dataset = pool.starmap(parallel_fn, args)
    print(f"Running time {n+1}-th round: {time.time() - start_time} seconds")
    
    flatten_round_dataset = []
    for batch in round_dataset:
        flatten_round_dataset += batch
    self_play_dataset += flatten_round_dataset
    
    trainer = Trainer(game = TicTacToe, mcts = MCTS, model=model)
    fit_history = trainer.train(self_play_dataset, n_epochs=50, batch_size=50, learning_rate=1e-2)
    
    trainer.model.save("saved_model/tmp_model")
    
    with open("selfplay_data.pickle", "wb") as fp:
        pickle.dump(self_play_dataset, fp, protocol=pickle.HIGHEST_PROTOCOL)
    